In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [816 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-13 22:43:39--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.86MB/s    in 0.2s    

2022-07-13 22:43:40 (4.86 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Final Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://plasticpollution.s3.amazonaws.com/Plastic_waste_mismanagement_clean.csv"
spark.sparkContext.addFile(url)
Plastic_waste_mismanagement_df = spark.read.csv(SparkFiles.get("Plastic_waste_mismanagement_clean.csv"), sep=",", header=True, inferSchema=True)

In [5]:
# Show DataFrame
Plastic_waste_mismanagement_df.show()    

+--------------------+----+----+-------------------------+
|             Country|Code|Year|PlasticWaste(metric tons)|
+--------------------+----+----+-------------------------+
|              Africa|null|2019|                 13690104|
|             Albania| ALB|2019|                    69833|
|             Algeria| DZA|2019|                   764578|
|              Angola| AGO|2019|                   236946|
| Antigua and Barbuda| ATG|2019|                      627|
|           Argentina| ARG|2019|                   465808|
|                Asia|null|2019|                 39881167|
|           Australia| AUS|2019|                     5266|
|             Bahamas| BHS|2019|                     2212|
|             Bahrain| BHR|2019|                     1043|
|          Bangladesh| BGD|2019|                  1021990|
|            Barbados| BRB|2019|                      872|
|             Belgium| BEL|2019|                     2284|
|              Belize| BLZ|2019|                     602

In [8]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://plasticpollution.s3.amazonaws.com/pbt_clean.csv"
spark.sparkContext.addFile(url)
pbt_df = spark.read.csv(SparkFiles.get("pbt_clean.csv"), sep=",", header=True, inferSchema=True)

In [9]:
# Show DataFrame
pbt_df.show()    

+-------------------+----+--------+
|       COUNTRY NAME|YEAR|   VALUE|
+-------------------+----+--------+
|            Albania|2017| 16481.0|
|            Albania|2018| 18094.0|
|            Albania|2019| 18500.0|
|            Albania|2020| 20157.0|
|            Albania|2021| 19828.0|
|            Algeria|2017|153459.0|
|            Algeria|2018|144421.0|
|            Algeria|2019|157950.0|
|            Algeria|2020|175154.0|
|            Algeria|2021|180214.0|
|             Angola|2017| 48632.0|
|             Angola|2018| 50225.0|
|             Angola|2019| 52877.0|
|             Angola|2020| 54660.0|
|             Angola|2021| 55728.0|
|Antigua and Barbuda|2017|   175.0|
|Antigua and Barbuda|2018|   185.0|
|Antigua and Barbuda|2019|   172.0|
|Antigua and Barbuda|2020|   144.0|
|Antigua and Barbuda|2021|   151.0|
+-------------------+----+--------+
only showing top 20 rows



In [10]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://plasticpollution.s3.amazonaws.com/MLW_meta_clean.csv"
spark.sparkContext.addFile(url)
mlw_meta_df = spark.read.csv(SparkFiles.get("MLW_meta_clean.csv"), sep=",", header=True, inferSchema=True )

In [11]:
#Show dataframe
mlw_meta_df.show()

+-----------+--------+--------------------+
|generalcode|category|         generalname|
+-----------+--------+--------------------+
|         G1| Plastic|4/6-pack yokes, s...|
|         G3| Plastic|Shopping Bags  in...|
|         G4| Plastic|Small plastic bag...|
|         G5| Plastic|Plastic bags coll...|
|         G7| Plastic|Drink bottles  <=...|
|         G8| Plastic|Drink bottles  >0.5l|
|         G9| Plastic|Cleaner bottles &...|
|        G10| Plastic|Food containers i...|
|        G11| Plastic|Beach use related...|
|        G12| Plastic|Other cosmetics b...|
|        G13| Plastic|Other bottles & c...|
|        G14| Plastic|Engine oil bottle...|
|        G15| Plastic|Engine oil bottle...|
|        G16| Plastic|Jerry cans (squar...|
|        G17| Plastic|Injection gun con...|
|        G18| Plastic|Crates and contai...|
|        G19| Plastic|           Car parts|
|        G21| Plastic|Plastic caps/lids...|
|        G22| Plastic|Plastic caps/lids...|
|        G23| Plastic|Plastic ca

In [12]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://plasticpollution.s3.amazonaws.com/MLW_data_clean.csv"
spark.sparkContext.addFile(url)
mlw_data_df = spark.read.csv(SparkFiles.get("MLW_data_clean.csv"), sep=",", header=True, inferSchema=True,)
#Show dataframe
mlw_data_df.show()

+------------+--------------------+--------------------+-------------+-------------+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+-----+----+-----+----+----+----+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|BeachCountry|           BeachName|    BeachRegionalSea|BeachLength_m|    BeachType|  G1|  G3|  G4|  G5|  G7|   G8|  G9| G10| G11| G12| G13| G14| G15| G16| G17| G18| G19|  G21| G22|  G23| G24| G25| G26|   G27| G28| G29| G30| G31| G32| G33| G34| G35| G36| G37| G40| G41| G42| G43| G44| G45| G46| G47| G49| G50| G52| G53|   G54| G56| G57| G58| G59| G60| G62| G63| G64| G65| G66| G67| G68| G69| G70| G71| G72|  G73|  G76| G77|  G79| G80| G82| G83| G84| G85| G86| G87| G88| G89| 

Connect to the AWS RDS instance and write each DataFrame to its table

In [13]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://plasticpollutiondb.c8tx5tieh7hn.us-east-1.rds.amazonaws.com:5432/plasticpollutiondb"
config = {"user":"plasticpollution",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [14]:
# Write Plastic_waste_mismanagement_df to table in RDS
Plastic_waste_mismanagement_df.write.jdbc(url=jdbc_url, table='plastic_waste_mismanagement', mode=mode, properties=config)

In [15]:
# Write pbt_df to table in RDS
pbt_df.write.jdbc(url=jdbc_url, table='pbt', mode=mode, properties=config)

In [16]:
# Write mlw_meta_df to table in RDS
mlw_meta_df.write.jdbc(url=jdbc_url, table='mlw_meta', mode=mode, properties=config)

In [17]:
# Write mlw_data_df to table in RDS
mlw_data_df.write.jdbc(url=jdbc_url, table='mlw_data', mode=mode, properties=config)